In [26]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = 'conv1d.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    print(path_to_file)
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    os.chdir(path_to_file)
    !pwd4/rwG2zPYz5nLw4oxBo_eyUAEiKbHKef-aAv0iLRHPbfc_HJc1tMyuWYo

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
./conv1d.ipynb

/bin/bash: pwd4/rwG2zPYz5nLw4oxBo_eyUAEiKbHKef-aAv0iLRHPbfc_HJc1tMyuWYo: No such file or directory


In [0]:
import torch
import torch.nn as nn
import numpy as np
import time
import torch.nn.functional as F
import random
from matplotlib import pyplot as plt

In [28]:
trainer1=np.load("trainData__SMOTE_all_10s_f0.npz","rb")
tester1=np.load("trainData__SMOTE_all_10s_f0_TEST.npz", "rb")

trainer2=np.load("trainData__SMOTE_all_10s_f1.npz","rb")
tester2=np.load("trainData__SMOTE_all_10s_f1_TEST.npz","rb")

x_train=torch.from_numpy(trainer2['x'][:])
y_train=torch.from_numpy(trainer2['y'][:])
y_train=y_train.long()

x_test=torch.from_numpy(tester2['x'][:])
y_test=torch.from_numpy(tester2['y'][:])
y_test=y_test.long()

print("Training data size= ",x_train.size(), "\t\tTraining label size= ",y_train.size(),sep="\t")
print("Testing data size= ", x_test.size(), "\t\tTesting label size= ", y_test.size(),sep="\t")

#REshape x_test and y_test into x_train and y_train dimensions

x_test=x_test.view(-1,3000)
y_test=y_test.view(-1,)

print("\n\nReshaped Testing data size= ", x_test.size(), "\t\tReshaped Testing label size= ", y_test.size(),sep="\t")

Training data size= 	torch.Size([55130, 3000])			Training label size= 	torch.Size([55130])
Testing data size= 	torch.Size([2032, 1, 3000])			Testing label size= 	torch.Size([2032, 1])


Reshaped Testing data size= 	torch.Size([2032, 3000])			Reshaped Testing label size= 	torch.Size([2032])


In [29]:
std= x_train.std()
mean=x_train.mean()

print(std,mean,sep="  ")

tensor(0.9682)  tensor(-1.1859e-09)


In [0]:
batch_size = 100
learning_rate = 0.01

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        
        #Pool and activation fn's here 
        activation=nn.ReLU()       
        pool_8 = nn.MaxPool1d(kernel_size=8,stride=8) #Size=8
        pool_4 = nn.MaxPool1d(kernel_size=4,stride=4) #Size=4
        pool_2 = nn.MaxPool1d(kernel_size=2,stride=2) #Size=2
        
        #Dropout
        self.drop_out = nn.Dropout()
        
        
        #ConvNet time
        conv1_t=nn.Conv1d (1, 64, kernel_size=50,stride=6,padding=1)
        self.conv2_t=nn.Conv1d(64,128,kernel_size=8,stride=1,padding=1)
        self.conv3_t=nn.Conv1d(128,128,kernel_size=8,stride=1,padding=1)
        conv4_t=nn.Conv1d(128,128,kernel_size=8,stride=1,padding=1)
        
        #Layers time
        self.layer1_t=nn.Sequential(conv1_t,activation,pool_8)
        self.layer2_t=nn.Sequential(conv4_t,activation,pool_4)
        
        #ConvNet freq
        
        conv1_f=nn.Conv1d(1,  64,  kernel_size=400, stride=50,padding=1)
        self.conv2_f=nn.Conv1d(64, 128, kernel_size=6,   stride=1,padding=1)
        self.conv3_f=nn.Conv1d(128,128, kernel_size=6,   stride=1,padding=1)
        conv4_f=nn.Conv1d(128,128, kernel_size=6,   stride=1,padding=1)
        
        
        #Layers freq
        
        self.layer1_f=nn.Sequential(conv1_f, activation, pool_4)
        self.layer2_f=nn.Sequential(conv4_f, activation, pool_2)
        
        self.layer=nn.Sequential(nn.Linear(1664,5,bias=False),activation)
        
        
        
        
   
    def forward(self,x):
        
        
        #Time here
        out_t=self.layer1_t(x)
        out_t=self.drop_out(out_t)  #DropOut
        out_t=self.conv2_t(out_t)
        out_t=self.conv3_t(out_t)
        out_t=self.layer2_t(out_t)
        
        #Freq here
        
        out_f=self.layer1_f(x)
        out_f=self.drop_out(out_f)  #DropOut
        out_f=self.conv2_f(out_f)
        out_f=self.conv3_f(out_f)
        out_f=self.layer2_f(out_f)
        
        out=torch.cat((out_t,out_f),2)  #Concat
        out=self.drop_out(out)   #DropOut
        out=out.view(-1,1664)
        
        out=self.layer(out)
        
        #print("Output size=  ",size)
       # print("modified= ",out.reshape(-1) )
        return out

In [31]:
net=ConvNet()
print(net)
net = net.to(torch.device("cuda"))

mean=mean.to(torch.device("cuda"))

std=std.to(torch.device("cuda"))
criterion = nn.CrossEntropyLoss()

ConvNet(
  (drop_out): Dropout(p=0.5, inplace=False)
  (conv2_t): Conv1d(64, 128, kernel_size=(8,), stride=(1,), padding=(1,))
  (conv3_t): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(1,))
  (layer1_t): Sequential(
    (0): Conv1d(1, 64, kernel_size=(50,), stride=(6,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2_t): Sequential(
    (0): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2_f): Conv1d(64, 128, kernel_size=(6,), stride=(1,), padding=(1,))
  (conv3_f): Conv1d(128, 128, kernel_size=(6,), stride=(1,), padding=(1,))
  (layer1_f): Sequential(
    (0): Conv1d(1, 64, kernel_size=(400,), stride=(50,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2_f): Sequential(
    (0): Conv1d(128, 1

In [0]:
def get_error(scores,labels):
  
  batch_size = scores.size(0)
  predicted_labels = scores.argmax(dim=1)
  indicator = (predicted_labels == labels)
  num_matches=indicator.sum()
  
  return 1-num_matches.float()/batch_size

In [0]:
def eval_on_test_set():

    running_error=0
    num_batches=0

    for i in range(0,1000,batch_size):

        minibatch_data =  x_test[i:i+batch_size].unsqueeze(dim=1)
        minibatch_label= y_test[i:i+batch_size]

        minibatch_data=minibatch_data.to(torch.device("cuda"))
        minibatch_label=minibatch_label.to(torch.device("cuda"))
        
        inputs = (minibatch_data - mean)/std    # ONLY CHANGE IS HERE!

        scores=net( inputs ) 

        error = get_error( scores , minibatch_label)

        running_error += error.item()

        num_batches+=1


    total_error = running_error/num_batches
    print( 'error rate on test set =', total_error*100 ,'percent')

In [34]:
start=time.time()
epoch_axis=list()
error_axis=list()

for epoch in range(1,40):
    
       
    optimizer=torch.optim.SGD( net.parameters() , lr=learning_rate )
        
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(40000)
 
    for count in range(0,10000,batch_size):
        
        # FORWARD AND BACKWARD PASS
    
        optimizer.zero_grad()
             
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  x_train[indices].unsqueeze(dim=1)
        minibatch_label=  y_train[indices]
        
        #print("minibatch_data= ",minibatch_data.size())
        #print("minibatch_label= ",minibatch_label.size())
        
        
        minibatch_data=minibatch_data.to(torch.device("cuda"))
        minibatch_label=minibatch_label.to(torch.device("cuda"))
        
        inputs = (minibatch_data - mean)/std      # ONLY CHANGE IS HERE!
        
        inputs.requires_grad_()
                
        scores=net( inputs ) 
        #print("SCores=", scores.size())

        loss =  criterion( scores , minibatch_label) 
          
        loss.backward()
        
        optimizer.step()
        

        # COMPUTE STATS
        
        running_loss += loss.detach().item()
        
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
                   
        
        
        
        num_batches+=1        
    
    
    # AVERAGE STATS THEN DISPLAY
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t learning rate=', learning_rate  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    epoch_axis.append(epoch)
    error_axis.append(error)
    eval_on_test_set() 
    print(' ')

#plt.plot(epoch_axis,error_axis)
#plt.show()

epoch= 1 	 time= 0.039295260111490884 min 	 learning rate= 0.01 	 loss= 1.465210382938385 	 error= 58.81000101566315 percent
error rate on test set = 42.000001072883606 percent
 
epoch= 2 	 time= 0.07844666639963786 min 	 learning rate= 0.01 	 loss= 1.4199770534038543 	 error= 58.63000148534775 percent
error rate on test set = 46.60000145435333 percent
 
epoch= 3 	 time= 0.11772299607594808 min 	 learning rate= 0.01 	 loss= 1.3081521010398864 	 error= 53.360000967979424 percent
error rate on test set = 52.2000002861023 percent
 
epoch= 4 	 time= 0.15712586641311646 min 	 learning rate= 0.01 	 loss= 1.1686065846681595 	 error= 46.30000102519989 percent
error rate on test set = 52.800002098083496 percent
 
epoch= 5 	 time= 0.19651205937067667 min 	 learning rate= 0.01 	 loss= 1.083027920126915 	 error= 41.62000125646591 percent
error rate on test set = 62.400002479553216 percent
 
epoch= 6 	 time= 0.2358357548713684 min 	 learning rate= 0.01 	 loss= 1.0234754973649978 	 error= 39.5300014

In [35]:
#Tester
'''
test_no = random.randint(0,1000)


test_data = x_test[test_no]
test_data = test_data.view(1,3000).unsqueeze(dim=1)
test_label= y_test[test_no]

test_data = test_data.to(torch.device("cuda"))
test_data = (test_data-mean)/std

#print(test_data.size())
scores = net(test_data)
#print(scores)
prob = F.softmax(scores, dim=1)

print(test_label)
print(max(prob))




'''

##index


for i in range(0,5):
  t=y_test.to("cpu")
  t=t.numpy()
  pass_val=np.where(t==i)[0][0]
  
  test_data = x_test[pass_val]
  test_data = test_data.view(1,3000).unsqueeze(dim=1)
  test_label= y_test[pass_val]
  
  test_data = test_data.to(torch.device("cuda"))
  test_data = (test_data-mean)/std
  
  scores = net(test_data)
  #print(scores)
  prob = F.softmax(scores, dim=1)
  prob1=prob.to("cpu")
  prob1=prob1.detach().numpy()

  val=prob1.argmax(axis=1)
  
  print("Test_label= ",test_label," predicted_label=", val,"Outcome= ",test_label==val[0])






Test_label=  tensor(0)  predicted_label= [0] Outcome=  tensor(True)
Test_label=  tensor(1)  predicted_label= [4] Outcome=  tensor(False)
Test_label=  tensor(2)  predicted_label= [2] Outcome=  tensor(True)
Test_label=  tensor(3)  predicted_label= [3] Outcome=  tensor(True)
Test_label=  tensor(4)  predicted_label= [0] Outcome=  tensor(False)
